<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/01-Pandas/B2a-Inserting_Data_in_MySQL_using_PythonPandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
!pip install -U -q PyMySQL sqlalchemy sql_magic

     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 1.6 MB 18.8 MB/s 
     |████████████████████████████████| 1.6 MB 37.3 MB/s 
     |████████████████████████████████| 120 kB 39.3 MB/s 
     |████████████████████████████████| 82 kB 435 kB/s 


In [2]:
import requests

In [3]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = "https://gbfs.citibikenyc.com/gbfs/en/station_status.json"

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json()

In [4]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [5]:
len(data)

1708

In [6]:
import pandas as pd

df = pd.DataFrame(data)
df.head(5)

,rental_methods,station_id,electric_bike_surcharge_waiver,rental_uris,name,has_kiosk,external_id,lat,region_id,capacity,short_name,legacy_id,eightd_station_services,lon,eightd_has_key_dispenser,station_type
0,"[KEY, CREDITCARD]",72,False,{'android': 'https://bkn.lft.to/lastmile_qr_sc...,W 52 St & 11 Ave,True,66db237e-0aca-11e7-82f6-3863bb44ef7c,40.767272,71,20,6926.01,72,[],-73.993929,False,classic
1,"[KEY, CREDITCARD]",79,False,{'android': 'https://bkn.lft.to/lastmile_qr_sc...,Franklin St & W Broadway,True,66db269c-0aca-11e7-82f6-3863bb44ef7c,40.719116,71,33,5430.08,79,[],-74.006667,False,classic
2,"[KEY, CREDITCARD]",82,False,{'android': 'https://bkn.lft.to/lastmile_qr_sc...,St James Pl & Pearl St,True,66db277a-0aca-11e7-82f6-3863bb44ef7c,40.711174,71,27,5167.06,82,[],-74.000165,False,classic
3,"[KEY, CREDITCARD]",83,False,{'android': 'https://bkn.lft.to/lastmile_qr_sc...,Atlantic Ave & Fort Greene Pl,True,66db281e-0aca-11e7-82f6-3863bb44ef7c,40.683826,71,62,4354.07,83,[],-73.976323,False,classic
4,"[KEY, CREDITCARD]",116,False,{'android': 'https://bkn.lft.to/lastmile_qr_sc...,W 17 St & 8 Ave,True,66db28b5-0aca-11e7-82f6-3863bb44ef7c,40.741776,71,74,6148.02,116,[],-74.001497,False,classic


In [7]:
# We drop the 'rental methods' columns,
# as they contains multiple values and
# we cannot insert lists in a database cell.
df.drop(
    ["rental_methods", "eightd_station_services", "rental_uris"],
    axis="columns",
    inplace=True,
)
df.head(5)

,station_id,electric_bike_surcharge_waiver,name,has_kiosk,external_id,lat,region_id,capacity,short_name,legacy_id,lon,eightd_has_key_dispenser,station_type
0,72,False,W 52 St & 11 Ave,True,66db237e-0aca-11e7-82f6-3863bb44ef7c,40.767272,71,20,6926.01,72,-73.993929,False,classic
1,79,False,Franklin St & W Broadway,True,66db269c-0aca-11e7-82f6-3863bb44ef7c,40.719116,71,33,5430.08,79,-74.006667,False,classic
2,82,False,St James Pl & Pearl St,True,66db277a-0aca-11e7-82f6-3863bb44ef7c,40.711174,71,27,5167.06,82,-74.000165,False,classic
3,83,False,Atlantic Ave & Fort Greene Pl,True,66db281e-0aca-11e7-82f6-3863bb44ef7c,40.683826,71,62,4354.07,83,-73.976323,False,classic
4,116,False,W 17 St & 8 Ave,True,66db28b5-0aca-11e7-82f6-3863bb44ef7c,40.741776,71,74,6148.02,116,-74.001497,False,classic


### Writing a Pandas Dataframe in a MySQL Table

In [8]:
import sqlalchemy
from sqlalchemy import create_engine

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

Once we have connected successfully, we need to create our database:

In [9]:
# Query to create a database
# In this example, we will try to create the (existing) database "public"
# But in general, we can give any name to the database
db_name = "public"
create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

# Create a database
engine.execute(create_db_query)

# And lets switch to the database
engine.execute(f"USE {db_name}")

In [10]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid

if "suffix" not in globals():
    suffix = str(uuid.uuid4())[:8]
print(suffix)

156f51ce


### Create Table and Store Data in Database using the `to_sql` command

Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [11]:
table_name = f"Stations_{suffix}"
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation

# This step is optional, but it is good practice to define explicitly the
# data types before storing things in a database. In many cases, this can be ommitted, though.

create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (legacy_id int,
                                station_id int,
                                region_id int,
                                external_id varchar(50),
                                lat float,
                                lon float,
                                short_name varchar(10),
                                name varchar(250),
                                station_type varchar(10),
                                capacity int,
                                electric_bike_surcharge_waiver bool,
                                eightd_has_key_dispenser bool,
                                has_kiosk bool,
                                PRIMARY KEY(station_id)
                                )"""
engine.execute(create_table_query)

df.to_sql(
    name=table_name,
    schema=db_name,
    con=engine,
    if_exists="append",
    index=False
)

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = f"ALTER TABLE {table_name} ADD PRIMARY KEY(station_id)"
print(add_key_query)
engine.execute(add_key_query)

### Reading from a SQL Database in Python using the `read_sql` command in Pandas

We can similarly read from the database using Pandas

In [12]:
query = f"SELECT * FROM {db_name}.{table_name}"
print(query)

SELECT * FROM public.Stations_156f51ce


In [13]:
df2 = pd.read_sql(query, con=engine)
df2.head(5)

,legacy_id,station_id,region_id,external_id,lat,lon,short_name,name,station_type,capacity,electric_bike_surcharge_waiver,eightd_has_key_dispenser,has_kiosk
0,72,72,71,66db237e-0aca-11e7-82f6-3863bb44ef7c,40.7673,-73.9939,6926.01,W 52 St & 11 Ave,classic,20,0,0,1
1,79,79,71,66db269c-0aca-11e7-82f6-3863bb44ef7c,40.7191,-74.0067,5430.08,Franklin St & W Broadway,classic,33,0,0,1
2,82,82,71,66db277a-0aca-11e7-82f6-3863bb44ef7c,40.7112,-74.0002,5167.06,St James Pl & Pearl St,classic,27,0,0,1
3,83,83,71,66db281e-0aca-11e7-82f6-3863bb44ef7c,40.6838,-73.9763,4354.07,Atlantic Ave & Fort Greene Pl,classic,62,0,0,1
4,116,116,71,66db28b5-0aca-11e7-82f6-3863bb44ef7c,40.7418,-74.0015,6148.02,W 17 St & 8 Ave,classic,74,0,0,1


### Export Data from Database to CSV or Excel

And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [ ]:
# The necessary library to write in Excel
# !sudo pip3 install -U openpyxl

In [ ]:
df2.to_excel("citibike.xlsx")
df2.to_csv("citibike.csv")

### Cleanup

Finally, let's clean up and delete the table that we created

In [14]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
print(drop_table_query)
engine.execute(drop_table_query)

DROP TABLE IF EXISTS public.Stations_156f51ce


### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.